In [3]:
import pandas as pd
import transformers
import torch
import random
import numpy as np
import pickle
from tqdm import tqdm
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('path_to_model')

/home/eshaan/Cross-task-ICL/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from utils.data import Prompt
prompter=Prompt()

In [6]:
import math
def create_embedding(data,model):
    o=model.encode(data, convert_to_tensor=True).detach().cpu()
    return o

def create_embeddings(dataset,prompter,id,model,print_example=False,batch_size=1024):
    
    num_batch=math.ceil(len(dataset)/batch_size)
    
    for batch_i in tqdm(range(num_batch),desc='Getting emb of batches'):
        
        batch_dataset=dataset[batch_i*batch_size:(batch_i+1)*batch_size]        
        batch_inputs=[prompter.get_input_data_without_def(d,id) for d in batch_dataset]
        emb=create_embedding(batch_inputs,model)
        for i,d in enumerate(batch_dataset):
            d['emb']=emb[i]
            if print_example:
                print(d)
                print_example=False

In [7]:
import os
path='data/source'
if not os.path.exists(path):
    os.makedirs(path)
path='data/target'
if not os.path.exists(path):
    os.makedirs(path)
path='data/target-unlabeled'
if not os.path.exists(path):
    os.makedirs(path)

In [8]:
import json

# Source

In [7]:
split='data/source'
k=10000

## Mnli

In [8]:
dataset_name='mnli'
dataset= load_dataset('glue',dataset_name)['train']
test_id=[]
test_s1=[]
test_s2=[]
test_l=[]
import random
label2text={
    2:'contradiction',
    1:'neutral',
    0:'entailment'
}

for d in dataset:
        test_id.append(d['idx'])
        test_s1.append(d['premise'])
        test_s2.append(d['hypothesis'])
        test_l.append(label2text[d['label']])

df={
    'id':test_id,
    'label':test_l,
    'sentence1':test_s1,
    'sentence2':test_s2
}
test_df=pd.DataFrame(df)
test_df=test_df.sample(k,random_state=0)
test_s_df=test_df.to_dict('records')
print(len(test_s_df))

Using the latest cached version of the module from /home/eshaan/.cache/huggingface/modules/datasets_modules/datasets/glue/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad (last modified on Fri Aug 11 17:12:00 2023) since it couldn't be found locally at glue., or remotely on the Hugging Face Hub.


10000


In [9]:
prompt_temp_id='001'
create_embeddings(test_s_df,prompter,prompt_temp_id,model)

Getting emb of batches: 100%|██████████| 10/10 [00:11<00:00,  1.11s/it]


In [10]:
data_to_save={
        'dataset_name':dataset_name,
        'prompt_temp_id':'001',
        'data':test_s_df,
        'set':'source',
        'instruction':' Given Sentence 1 which is a premise and Sentence 2 which is a hypothesis do natural language inference on the pair. In natural language inference we mark whether the premise and hypothesis are "neutral", "contradiction" or "entailment". The pair are said to be "entailed" if the premise justifies/supports the hypothesis, if the pair contradict each other we label them as "contradiction" and label them "neutral" in all other cases'
}

with open(f'{split}/{dataset_name}_train.pkl', 'wb') as openfile:
        # Reading from json file
        pickle.dump(data_to_save, openfile)
with open(f'{split}/{dataset_name}_train.pkl','rb') as f:
        test_set=pickle.load(f)

In [11]:
test_set['data'][0]

{'id': 348280,
 'label': 'contradiction',
 'sentence1': 'Before any legal action is brought against this rule in federal court, the administrative appeal rights set forth at 7 C.F.R.',
 'sentence2': 'Before we send this to federal court, there is a conference.',
 'emb': tensor([-2.5739e-02,  3.1649e-02,  3.0162e-03,  3.0356e-02,  1.2583e-02,
          1.1141e-01,  1.2851e-02,  4.5885e-03,  3.2056e-02,  8.5702e-02,
          3.3429e-02,  8.6360e-02, -4.3918e-02,  7.0449e-03,  4.8251e-02,
         -6.9651e-03,  6.3258e-02, -1.4582e-02, -3.4724e-02,  3.6546e-02,
          2.1035e-02,  1.0543e-02, -2.8848e-02,  2.6292e-02, -2.5695e-02,
         -6.1129e-02, -1.0759e-01, -4.9241e-02,  5.7713e-02, -2.5417e-02,
         -2.5361e-02,  1.2542e-02,  1.1923e-01,  5.1684e-02,  3.3623e-02,
         -2.4713e-03,  4.2015e-02, -1.8845e-02,  3.5000e-02,  4.7962e-02,
         -2.8507e-02, -7.7633e-02, -1.7560e-02, -3.3662e-02, -4.7150e-02,
          7.9259e-02,  2.2829e-03,  2.2897e-02, -1.6318e-02,  4.

## QQP

In [12]:
dataset_name='qqp'
dataset= load_dataset('glue',dataset_name)['train']
test_id=[]
test_s1=[]
test_s2=[]
test_l=[]
import random
label2text={
    1:'duplicate',
    0:'not duplicate'
}

for d in dataset:
        test_id.append(d['idx'])
        test_s1.append(d['question1'])
        test_s2.append(d['question2'])
        test_l.append(label2text[d['label']])

df={
    'id':test_id,
    'label':test_l,
    'sentence1':test_s1,
    'sentence2':test_s2
}
test_df=pd.DataFrame(df)
test_df=test_df.sample(k,random_state=0)
test_s_df=test_df.to_dict('records')
print(len(test_s_df))

Using the latest cached version of the module from /home/eshaan/.cache/huggingface/modules/datasets_modules/datasets/glue/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad (last modified on Fri Aug 11 17:12:00 2023) since it couldn't be found locally at glue., or remotely on the Hugging Face Hub.


10000


In [13]:
prompt_temp_id='002'
create_embeddings(test_s_df,prompter,prompt_temp_id,model)

Getting emb of batches: 100%|██████████| 10/10 [00:05<00:00,  1.77it/s]


In [14]:
data_to_save={
        'dataset_name':dataset_name,
        'prompt_temp_id':'002',
        'data':test_s_df,
        'instruction':'Given two question pairs do text classification based on whether they are duplicates or not. The questions are mined from the popular online discussion forum Quora. As duplicate quetion might be present on Quora, the task is to label two identical questions as "duplicate" if they ask the same query else label the pair as "not duplicate".',
        'set':'source'
        }

with open(f'{split}/{dataset_name}_train.pkl', 'wb') as openfile:
        # Reading from json file
        pickle.dump(data_to_save, openfile)

## Boolq

In [15]:
dataset_name='boolq'
dataset= load_dataset(dataset_name)['train']
test_id=[]
test_s=[]
test_c=[]
test_l=[]
id=1
import random


for d in dataset:
        test_id.append(id)
        id+=1
        test_s.append(d['question'])
        test_c.append(d['passage'])
        test_l.append(str(d['answer']))

df={
    'id':test_id,
    'label':test_l,
    'sentence':test_s,
    'context':test_c
}
test_df=pd.DataFrame(df)
test_s_df=test_df.to_dict('records')
print(len(test_s_df))

Using the latest cached version of the module from /home/eshaan/.cache/huggingface/modules/datasets_modules/datasets/boolq/bf0dd57da941c50de94ae3ce3cef7fea48c08f337a4b7aac484e9dddc5aa24e5 (last modified on Fri Aug 11 17:16:37 2023) since it couldn't be found locally at boolq., or remotely on the Hugging Face Hub.


9427


In [16]:
prompt_temp_id='003'
create_embeddings(test_s_df,prompter,prompt_temp_id,model)

Getting emb of batches: 100%|██████████| 10/10 [00:09<00:00,  1.00it/s]


In [17]:
data_to_save={
        'dataset_name':dataset_name,
        'prompt_temp_id':'003',
        'data':test_s_df,
        'instruction':'Given a context and a question do binary true and false type text classification. You are given a passage as context and a question related to the passage that can be answered as "True" or "False". Based on the context, question and your reasoning ability answer in a "True" and "False".',
        'set':'source'
        }

with open(f'{split}/{dataset_name}_train.pkl', 'wb') as openfile:
        # Reading from json file
        pickle.dump(data_to_save, openfile)

## Colll_NER

In [18]:
dataset_name='conll2003'
dataset= load_dataset(dataset_name)['train']
test_id=[]
test_s=[]
test_l=[]
id=1
import random

label2text={0:'O', 
1:'B-PER', 
2:'I-PER',
3:'B-ORG',
4:'I-ORG',
5:'B-LOC',
6:'I-LOC',
7:'B-MISC',
8:'I-MISC'
}

for d in dataset:

    l=[]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
    test_id.append(d['id'])
    for i in d['ner_tags']:
        l.append(label2text[i])
    test_s.append(' '.join(d['tokens']))
    test_l.append(' '.join(l))

df={
    'id':test_id,
    'label':test_l,
    'sentence':test_s
}
test_df=pd.DataFrame(df)
test_df=test_df.sample(k,random_state=0)
test_s_df=test_df.to_dict('records')
print(len(test_s_df))

Using the latest cached version of the module from /home/eshaan/.cache/huggingface/modules/datasets_modules/datasets/conll2003/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98 (last modified on Fri Aug 11 17:27:41 2023) since it couldn't be found locally at conll2003., or remotely on the Hugging Face Hub.


10000


In [19]:
prompt_temp_id='004'
create_embeddings(test_s_df,prompter,prompt_temp_id,model)

Getting emb of batches: 100%|██████████| 10/10 [00:05<00:00,  1.88it/s]


In [20]:
data_to_save={
        'dataset_name':dataset_name,
        'prompt_temp_id':'004',
        'data':test_s_df,
        'instruction':'Given a sentence do token classification on it seek to locate and classify named entities mentioned in the sentence provided. The pre-defined named entity categories along with there labeles are Person (PER), Location (LOC), Organization (ORG) and Miscellaneous (MIS). If the token is not an entity mark it as None. As the entity is more than two tokens long use the prefix B with the named entity token to represent the beginning and  use the prefix I till the entity ends.',
        'set':'source'
        }

with open(f'{split}/{dataset_name}_ner_train.pkl', 'wb') as openfile:
        # Reading from json file
        pickle.dump(data_to_save, openfile)

## Conll-POS

In [21]:
dataset_name='conll2003'
dataset= load_dataset(dataset_name)['train']
test_id=[]
test_s=[]
test_l=[]
id=1
import random

label={'"': 0, "''": 1, '#': 2, '$': 3, '(': 4, ')': 5, ',': 6, '.': 7, ':': 8, '``': 9, 'CC': 10, 'CD': 11, 'DT': 12,
 'EX': 13, 'FW': 14, 'IN': 15, 'JJ': 16, 'JJR': 17, 'JJS': 18, 'LS': 19, 'MD': 20, 'NN': 21, 'NNP': 22, 'NNPS': 23,
 'NNS': 24, 'NN|SYM': 25, 'PDT': 26, 'POS': 27, 'PRP': 28, 'PRP$': 29, 'RB': 30, 'RBR': 31, 'RBS': 32, 'RP': 33,
 'SYM': 34, 'TO': 35, 'UH': 36, 'VB': 37, 'VBD': 38, 'VBG': 39, 'VBN': 40, 'VBP': 41, 'VBZ': 42, 'WDT': 43,
 'WP': 44, 'WP$': 45, 'WRB': 46}


label2text=dict([(v,k) for k,v in label.items()])


for d in dataset:

    l=[]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
    test_id.append(d['id'])
    for i in d['pos_tags']:
        l.append(label2text[i])
    test_s.append(' '.join(d['tokens']))
    test_l.append(' '.join(l))

df={
    'id':test_id,
    'label':test_l,
    'sentence':test_s
}
test_df=pd.DataFrame(df)
test_df=test_df.sample(k,random_state=0)
test_s_df=test_df.to_dict('records')
print(len(test_s_df))

Using the latest cached version of the module from /home/eshaan/.cache/huggingface/modules/datasets_modules/datasets/conll2003/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98 (last modified on Fri Aug 11 17:27:41 2023) since it couldn't be found locally at conll2003., or remotely on the Hugging Face Hub.


10000


In [22]:
prompt_temp_id='004'
create_embeddings(test_s_df,prompter,prompt_temp_id,model)

Getting emb of batches: 100%|██████████| 10/10 [00:05<00:00,  1.88it/s]


In [23]:
data_to_save={
        'dataset_name':dataset_name,
        'prompt_temp_id':'004',
        'data':test_s_df,
        'instruction':'Given a sentence do token classification by doing Part-of-speech (POS) tagging, which is a process in natural language processing (NLP) where each word in a text is labeled with its corresponding part of speech. This can include nouns, verbs, adjectives, and other grammatical categories.',
        'set':'source'
        }

with open(f'{split}/{dataset_name}_pos_train.pkl', 'wb') as openfile:
        # Reading from json file
        pickle.dump(data_to_save, openfile)

## Commonsense_QA

In [24]:
dataset_name='commonsense_qa'
dataset= load_dataset('commonsense_qa')['validation']
test_id=[]
test_s=[]
test_l=[]

text2label={
    'A':0,
    'B':1,
    'C':2,
    'D':3,
    'E':4
}

label2text={
    0:'\nA. ',
    1:'\nB. ',
    2:'\nC. ',
    3:'\nD. ',
    4:'\nE. '
}

for d in dataset:
    test_id.append(d['id'])
    test_l.append(d['answerKey'])

    q=d['question']
    for i,a in enumerate(d['choices']['text']):
        q+=' '+label2text[i]+a

    test_s.append(q)

df={
    'id':test_id,
    'label':test_l,
    'sentence':test_s
}

test_df=pd.DataFrame(df)
test_s_df=test_df.to_dict('records')

Using the latest cached version of the module from /home/eshaan/.cache/huggingface/modules/datasets_modules/datasets/commonsense_qa/28d68f56649a7f0c23bc68eae850af914aa03f95f810011ae8cf58cc5ff5051b (last modified on Mon Sep 11 14:41:55 2023) since it couldn't be found locally at commonsense_qa., or remotely on the Hugging Face Hub.


In [25]:
prompt_temp_id='005'
create_embeddings(test_s_df,prompter,prompt_temp_id,model)

Getting emb of batches: 100%|██████████| 2/2 [00:00<00:00,  2.85it/s]


In [26]:
data_to_save={
        'dataset_name':dataset_name,
        'prompt_temp_id':'005',
        'data':test_s_df,
        'instruction':'The following task relates to commonsense reasoning. It consists of a question that can be easily solved using logical abilities and reasoning, a set of five options  "A.", "B.", "C.", "D." and "E." are also provided along with the question, one of these options answers the question logically. Use your reasoning ability to select the most appropriate answer from the provided choices "A.", "B.", "C.", "D." and "E." and assign these choices (i.e  "A.", "B.", "C.", "D." and "E.") as the label',  
        'set':'source'
        }

with open(f'{split}/{dataset_name}_train.pkl', 'wb') as openfile:
        # Reading from json file
        pickle.dump(data_to_save, openfile)

## ARC-Easy

In [27]:
dataset_name='ARC-Easy'
dataset= load_dataset('ai2_arc',dataset_name)['train']
test_id=[]
test_s=[]
test_l=[]
id=1
import random

label2text={
    'A':'A',
    'B':'B',
    "C":"C",
    "D":"D",
    '1':'A',
    '2':'B',
    "3":"C",
    "4":"D",
}

for d in dataset:
    if len(d['choices']['text'])!=4:
        continue
    test_id.append(d['id'])
    id+=1
    test_s.append(d['question']+'\nA. '+d['choices']['text'][0]+'\nB. '+d['choices']['text'][1]+'\nC. '+d['choices']['text'][2]+'\nD. '+d['choices']['text'][3])
    test_l.append(label2text[d['answerKey']])

df={
    'id':test_id,
    'label':test_l,
    'sentence':test_s
}
test_df=pd.DataFrame(df)
test_s_df=test_df.to_dict('records')
print(len(test_s_df))

Using the latest cached version of the module from /home/eshaan/.cache/huggingface/modules/datasets_modules/datasets/ai2_arc/1569c2591ea2683779581d9fb467203d9aa95543bb9b75dcfde5da92529fd7f6 (last modified on Fri Aug 11 17:17:10 2023) since it couldn't be found locally at ai2_arc., or remotely on the Hugging Face Hub.


2241


In [28]:
prompt_temp_id='005'
create_embeddings(test_s_df,prompter,prompt_temp_id,model)

Getting emb of batches: 100%|██████████| 3/3 [00:01<00:00,  2.03it/s]


In [29]:
data_to_save={
        'dataset_name':dataset_name,
        'prompt_temp_id':'005',
        'data':test_s_df,
        'instruction':'Given a question answering task from the 3rd to 9th-grade science exam. The question contains four options "A.", "B.", "C." and "D." Select the most appropriate choice that answers the question',
        'set':'source'
        }

with open(f'{split}/{dataset_name}_train.pkl', 'wb') as openfile:
        # Reading from json file
        pickle.dump(data_to_save, openfile)

## Race

In [30]:
dataset_name='race'
dataset= load_dataset('race','all')['train']
test_id=[]
test_s=[]
test_c=[]
test_l=[]
import random


for d in dataset:

    q=d['question']+' \nA. '+d['options'][0]+' \nB. '+d['options'][1]+' \nC. '+d['options'][2]+' \nD. '+d['options'][3]

    test_id.append(d['example_id'])
    test_s.append(q)
    test_c.append(d['article'])
    test_l.append(d['answer'])

df={
    'id':test_id,
    'label':test_l,
    'sentence':test_s,
    'context':test_c
}
test_df=pd.DataFrame(df)
test_df=test_df.sample(k,random_state=0)
test_s_df=test_df.to_dict('records')
print(len(test_s_df))

Using the latest cached version of the module from /home/eshaan/.cache/huggingface/modules/datasets_modules/datasets/race/5839ff74a429622f5f20cca69c5fcf0e87ac6d5fd2777c42b948000684829f7b (last modified on Fri Aug 11 17:33:56 2023) since it couldn't be found locally at race., or remotely on the Hugging Face Hub.


10000


In [31]:
prompt_temp_id='003'
create_embeddings(test_s_df,prompter,prompt_temp_id,model)

Getting emb of batches: 100%|██████████| 10/10 [00:16<00:00,  1.61s/it]


In [32]:
data_to_save={
        'dataset_name':dataset_name,
        'prompt_temp_id':'003',
        'data':test_s_df,
        'instruction':'Given a reading comprehension type question-answering from an english exam for school students. You are given a context and multiple choice question containing four options "A.", "B.", "C." and "D.". The question is answerable from the comprehension. Based on the question, the option and the context select the most appropriate answer from the provided choices "A.", "B.", "C." and "D.".',                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
        'set':'source'
        }

with open(f'{split}/{dataset_name}_train.pkl', 'wb') as openfile:
        # Reading from json file
        pickle.dump(data_to_save, openfile)

## AG_news

In [33]:
dataset_name='ag_news'
dataset= load_dataset(dataset_name)['train']
test_id=[]
test_s=[]
test_l=[]
id=1
import random

label2text={
        0:'world',
        1:'sports',
        2:'business',
        3:'technology'
}


for d in dataset:

    test_id.append(id)
    id+=1
    test_s.append(d['text'])
    test_l.append(label2text[d['label']])

df={
    'id':test_id,
    'label':test_l,
    'sentence':test_s
}
test_df=pd.DataFrame(df)
test_df=test_df.sample(k,random_state=0)
test_s_df=test_df.to_dict('records')
print(len(test_s_df))

Using the latest cached version of the module from /home/eshaan/.cache/huggingface/modules/datasets_modules/datasets/ag_news/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548 (last modified on Fri Aug 11 17:28:02 2023) since it couldn't be found locally at ag_news., or remotely on the Hugging Face Hub.


10000


In [34]:
prompt_temp_id='004'
create_embeddings(test_s_df,prompter,prompt_temp_id,model)

Getting emb of batches: 100%|██████████| 10/10 [00:06<00:00,  1.59it/s]


In [35]:
data_to_save={
        'dataset_name':dataset_name,
        'prompt_temp_id':'004',
        'data':test_s_df,
        'instruction':'Given a sentence do text classification, the sentence is a clipping from a news article that may be either related to sports, business, technology, or world news. You are to recognize the category of the sentence and label them as "sports", "business", "technology" or "world" news',
        'set':'source'
        }

with open(f'{split}/{dataset_name}_train.pkl', 'wb') as openfile:
        # Reading from json file
        pickle.dump(data_to_save, openfile)

## SST2

In [36]:
dataset_name='sst2'
dataset= load_dataset('glue','sst2')['train']
test_id=[]
test_s=[]
test_l=[]
import random
label2text={
    1:'positive',
    0:'negative'
}

for d in dataset:
        test_id.append(d['idx'])
        test_s.append(d['sentence'])
        test_l.append(label2text[d['label']])

df={
    'id':test_id,
    'label':test_l,
    'sentence':test_s
}
test_df=pd.DataFrame(df)
test_df=test_df.sample(k,random_state=0)
test_s_df=test_df.to_dict('records')
print(len(test_s_df))

Using the latest cached version of the module from /home/eshaan/.cache/huggingface/modules/datasets_modules/datasets/glue/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad (last modified on Fri Aug 11 17:12:00 2023) since it couldn't be found locally at glue., or remotely on the Hugging Face Hub.


10000


In [37]:
prompt_temp_id='004'
create_embeddings(test_s_df,prompter,prompt_temp_id,model)

Getting emb of batches: 100%|██████████| 10/10 [00:05<00:00,  1.95it/s]


In [38]:
data_to_save={
        'dataset_name':dataset_name,
        'prompt_temp_id':'004',
        'data':test_s_df,
        'instruction':'Given a movie review do text classification, based on the sentiment conveyed by the review label it as "positive" or "negative"',
        'set':'source'
        }

with open(f'{split}/{dataset_name}_train.pkl', 'wb') as openfile:
        # Reading from json file
        pickle.dump(data_to_save, openfile)

# Target

In [39]:
import numpy as np

In [40]:
k=500
split='data/target'

In [41]:
def create_new_dataset(data_list, N = k):
    
    selected_samples = []

    if type(data_list[0]['label']) == list:
        labels = list(set([example['label'][0] for example in data_list]))

        print(f"Label Space: {labels}")
        count = {}
        for label in labels:
            count[label] = 0

        np.random.seed(0)
        chosen_indices = np.random.randint(low = 0, high = len(data_list), size = len(data_list))
    
        for chosen in chosen_indices:
            if (chosen not in selected_samples) and (count[data_list[chosen]['label'][0]] < (N/(len(count)))):
                selected_samples.append(chosen)
                count[data_list[chosen]['label'][0]] +=1

        selected_samples.sort()

        print(f"Count of selected labels: {count}")

        selected_data = list(np.array(data_list)[selected_samples])
        return selected_data, labels

    else:
        labels = list(set([example['label'] for example in data_list]))

        print(f"Label Space: {labels}")
        count = {}
        for label in labels:
            count[label] = 0

        np.random.seed(0)
        chosen_indices = np.random.randint(low = 0, high = len(data_list), size = len(data_list))

        for chosen in chosen_indices:

            if (chosen not in selected_samples) and (count[data_list[chosen]['label']] < (N/(len(count)))):
                selected_samples.append(chosen)
                count[data_list[chosen]['label']] +=1


        selected_samples.sort()
        print(f"Count of selected labels: {count}")
        selected_data = list(np.array(data_list)[selected_samples])
        return selected_data, labels

## Med MCQ

In [42]:
dataset_name = 'medmcqa'
dataset= load_dataset(dataset_name)['validation']
test_id=[]
test_s=[]
test_l=[]

text2label={
    0:'A',
    1:'B',
    2:'C',
    3:'D'
}

for d in dataset:
    test_id.append(d['id'])
    test_l.append(text2label[d['cop']])

    q=d['question']+' \nA. '+d['opa']+' \nB. '+d['opb']+' \nC. '+d['opc']+' \nD. '+d['opd']

    test_s.append(q)

df={
    'id':test_id,
    'label':test_l,
    'sentence':test_s
}
test_df=pd.DataFrame(df)
test_s_df=test_df.to_dict('records')
print(len(test_s_df))

Using the latest cached version of the module from /home/eshaan/.cache/huggingface/modules/datasets_modules/datasets/medmcqa/f2fdfa9ccfbf9d148c0639e6afe3379f3c7e95c4d52d5e68ec1156e5004bd880 (last modified on Thu Sep 14 11:39:40 2023) since it couldn't be found locally at medmcqa., or remotely on the Hugging Face Hub.


4183


In [43]:
selected_data, labels = create_new_dataset(test_s_df)
prompt_temp_id='005'
create_embeddings(selected_data,prompter,prompt_temp_id,model)
data_to_save={
          'dataset_name': dataset_name,
          'prompt_temp_id':'005',
          'data': selected_data,
          'instruction': 'Given a multiple choice question containing four options "A.", "B.", "C." and "D." from a medical entrance exam. The question is related to a sub-field of medical science like Microbiology, Radiology, Ophthalmology, Surgery, Human anatomy, etc. Based on the question, the option and your knowledge of the medical field select the most appropriate answer from the provided choices "A.", "B.", "C." and "D.".',
          'labels': labels,
          'set':'target'
        }

with open(f'{split}/{dataset_name}_test.pkl', 'wb') as openfile:
        # Reading from json file
        pickle.dump(data_to_save, openfile)

Label Space: ['C', 'D', 'A', 'B']
Count of selected labels: {'C': 125, 'D': 125, 'A': 125, 'B': 125}


Getting emb of batches: 100%|██████████| 1/1 [00:00<00:00,  3.04it/s]


## SciQ

In [44]:
dataset_name = 'sciq'
dataset= load_dataset(dataset_name)['test']
test_id=[]
test_s=[]
test_l=[]
id=1
import random
text2label={
    'A':'distractor3',
    'B':'distractor1',
    'C':'distractor2',
    'D':'correct_answer'
}

for d in dataset:

    k=['A','B','C','D']
    k_=['A','B','C','D']
    random.shuffle(k)
    q=d['question']   
    for i,l in zip(k,k_):
        op=text2label[i]
        if op=='correct_answer':
            test_l.append(l)

        q+=f'\n{l}. '+d[op]
                    



    test_id.append(id)
    id+=1
    test_s.append(q)

df={
    'id':test_id,
    'label':test_l,
    'sentence':test_s
}
test_df=pd.DataFrame(df)
test_s_df=test_df.to_dict('records')
print(len(test_s_df))

Using the latest cached version of the module from /home/eshaan/.cache/huggingface/modules/datasets_modules/datasets/sciq/50e5c6e3795b55463819d399ec417bfd4c3c621105e00295ddb5f3633d708493 (last modified on Thu Sep 14 11:41:12 2023) since it couldn't be found locally at sciq., or remotely on the Hugging Face Hub.


1000


In [45]:
selected_data, labels = create_new_dataset(test_s_df)
prompt_temp_id='005'
create_embeddings(selected_data,prompter,prompt_temp_id,model)
data_to_save={
          'dataset_name': dataset_name,
          'prompt_temp_id':'005',
          'data': selected_data,
          'instruction': 'Given a question from a scientific exam about Physics, Chemistry, and Biology, among others. The question is in multiple choice format with four answer options "A.", "B.", "C." and "D.". Using your knowledge about the scientific fields answer the question and provide the label "A", "B", "C" and "D" as answer',
          'labels': labels,
          'set':'target'

}

with open(f'{split}/{dataset_name}_test.pkl', 'wb') as openfile:
        # Reading from json file
        pickle.dump(data_to_save, openfile)

Label Space: ['C', 'D', 'A', 'B']
Count of selected labels: {'C': 125, 'D': 125, 'A': 125, 'B': 125}


Getting emb of batches: 100%|██████████| 1/1 [00:00<00:00,  3.24it/s]


## ARC-Challenge

In [47]:
dataset_name='ARC-Challenge'
dataset= load_dataset('ai2_arc',dataset_name)['test']
test_id=[]
test_s=[]
test_l=[]
id=1
import random

label2text={
    'A':'A',
    'B':'B',
    "C":"C",
    "D":"D",
    '1':'A',
    '2':'B',
    "3":"C",
    "4":"D",
}

for d in dataset:
    if len(d['choices']['text'])!=4:
        continue
    test_id.append(d['id'])
    id+=1
    test_s.append(d['question']+'\nA. '+d['choices']['text'][0]+'\nB. '+d['choices']['text'][1]+'\nC. '+d['choices']['text'][2]+'\nD. '+d['choices']['text'][3])
    test_l.append(label2text[d['answerKey']])

df={
    'id':test_id,
    'label':test_l,
    'sentence':test_s
}
test_df=pd.DataFrame(df)
test_s_df=test_df.to_dict('records')
print(len(test_s_df))

Using the latest cached version of the module from /home/eshaan/.cache/huggingface/modules/datasets_modules/datasets/ai2_arc/1569c2591ea2683779581d9fb467203d9aa95543bb9b75dcfde5da92529fd7f6 (last modified on Fri Aug 11 17:17:10 2023) since it couldn't be found locally at ai2_arc., or remotely on the Hugging Face Hub.


1165


In [48]:
selected_data, labels = create_new_dataset(test_s_df)
prompt_temp_id='005'
create_embeddings(test_s_df,prompter,prompt_temp_id,model)
data_to_save={
        'dataset_name':dataset_name,
        'prompt_temp_id':'005',
        'data':selected_data,
        'instruction':'Given a question answering task from the 3rd to 9th-grade science exam. The question contains four options "A.", "B.", "C." and "D." Select the most appropriate choice that answers the question',
        'labels': labels,
        'set':'target'
        }

with open(f'{split}/{dataset_name}_test.pkl', 'wb') as openfile:
        # Reading from json file
        pickle.dump(data_to_save, openfile)

Label Space: ['C', 'D', 'A', 'B']
Count of selected labels: {'C': 125, 'D': 125, 'A': 125, 'B': 125}


Getting emb of batches:   0%|          | 0/2 [00:00<?, ?it/s]

Getting emb of batches: 100%|██████████| 2/2 [00:00<00:00,  2.31it/s]


In [49]:
selected_data[0]

{'id': 'Mercury_7175875',
 'label': 'C',
 'sentence': 'An astronomer observes that a planet rotates faster after a meteorite impact. Which is the most likely effect of this increase in rotation?\nA. Planetary density will decrease.\nB. Planetary years will become longer.\nC. Planetary days will become shorter.\nD. Planetary gravity will become stronger.',
 'emb': tensor([ 4.4570e-02,  2.2817e-02,  8.0172e-02, -3.3356e-02,  7.6208e-02,
          1.4206e-02, -1.6502e-02,  3.1332e-02,  5.2658e-02,  3.3354e-02,
          3.7082e-02,  3.7466e-02, -1.6066e-02, -1.7431e-02,  2.3428e-02,
          1.5800e-02, -1.8905e-02,  4.8023e-02, -8.2188e-02,  5.7523e-02,
         -3.9804e-02, -6.0829e-02, -5.9459e-02,  1.1388e-01,  6.3163e-02,
          2.1221e-02, -4.0567e-03,  3.1933e-02, -8.7232e-03, -3.6615e-02,
         -1.3328e-02,  3.0853e-02, -5.9815e-03, -1.5620e-02, -7.4276e-02,
         -4.2818e-02, -1.4294e-02, -1.8323e-02, -7.9383e-02, -4.2442e-02,
          1.5674e-03, -6.0584e-02,  7.6698e

## Social_i_qa

In [50]:
dataset_name = 'social_i_qa'
dataset= load_dataset(dataset_name)['validation']
test_id=[]
test_s=[]
test_l=[]
test_c=[]
id=1

text2label={
    '1':'A',
    '2':'B',
    '3':'C'
}
label2text={
    1:'answerA',
    2:'answerB',
    3:'answerC'
}


for d in dataset:
          test_id.append(id)
          id+=1

          test_l.append(text2label[d['label']])
          test_c.append(d['context'])

          q=d['question']+' \nA. '+d['answerA']+' \nB. '+d['answerB']+' \nC. '+d['answerC']
          test_s.append(q)

df={
    'id':test_id,
    'label':test_l,
    'sentence':test_s,
    'context':test_c
}
test_df=pd.DataFrame(df)
test_s_df=test_df.to_dict('records')
print(len(test_s_df))

Using the latest cached version of the module from /home/eshaan/.cache/huggingface/modules/datasets_modules/datasets/social_i_qa/674d85e42ac7430d3dcd4de7007feaffcb1527c535121e09bab2803fbcc925f8 (last modified on Thu Sep 14 11:41:32 2023) since it couldn't be found locally at social_i_qa., or remotely on the Hugging Face Hub.


1954


In [51]:
selected_data, labels = create_new_dataset(test_s_df)

prompt_temp_id='003'
create_embeddings(test_s_df,prompter,prompt_temp_id,model)
data_to_save={
          'dataset_name':dataset_name,
          'prompt_temp_id':'003',
          'data': selected_data,
          'instruction': 'Given an action as the context and a related question, you are to answer the question based on the context using your social intelligence. The question is of multiple choice form with three options "A", "B" and "C". Select the most appropriate answer from the provided choices "A", "B" and "C".',
          'labels': labels,
          'set':'target'
          
          }

with open(f'{split}/{dataset_name}_test.pkl', 'wb') as openfile:
        # Reading from json file
        pickle.dump(data_to_save, openfile)

Label Space: ['C', 'A', 'B']
Count of selected labels: {'C': 167, 'A': 167, 'B': 167}


Getting emb of batches:   0%|          | 0/2 [00:00<?, ?it/s]

Getting emb of batches: 100%|██████████| 2/2 [00:01<00:00,  1.89it/s]


## Financial Pharasebank

In [53]:
dataset_name='financial_phrasebank'
dataset= load_dataset(dataset_name,'sentences_allagree')['train']
dataset=dataset.train_test_split(test_size=0.2, stratify_by_column="label")
dataset=dataset['train']
test_id=[]
test_s=[]
test_l=[]
id=1

text2label={
    1:'neutral',
    2:'positive',
    0:'negative'
}

for d in dataset:
          test_id.append(id)
          id+=1
          test_s.append(d['sentence'])
          test_l.append(text2label[d['label']])

df={
    'id':test_id,
    'label':test_l,
    'sentence':test_s
}
test_df=pd.DataFrame(df)
test_s_df=test_df.to_dict('records')
print(len(test_s_df))

Using the latest cached version of the module from /home/eshaan/.cache/huggingface/modules/datasets_modules/datasets/financial_phrasebank/550bde12e6c30e2674da973a55f57edde5181d53f5a5a34c1531c53f93b7e141 (last modified on Fri Aug 11 17:37:50 2023) since it couldn't be found locally at financial_phrasebank., or remotely on the Hugging Face Hub.


2264


In [54]:
selected_data, labels = create_new_dataset(test_s_df)

prompt_temp_id='004'
create_embeddings(test_s_df,prompter,prompt_temp_id,model)
data_to_save={
        'dataset_name': dataset_name,
        'prompt_temp_id':'004',
        'data': selected_data,
        'instruction': 'Given a sentence mined from a financial news article, you are to determine the sentiment polarity of the sentence. The task deals with financial sentiment analysis. Based on the sentiment conveyed by the sentence, label the sentence as "negative", "positive" or "neutral"',
        'labels': labels,
        'set':'target'

}

with open(f'{split}/{dataset_name}_test.pkl', 'wb') as openfile:
        # Reading from json file
        pickle.dump(data_to_save, openfile)

Label Space: ['negative', 'positive', 'neutral']
Count of selected labels: {'negative': 167, 'positive': 167, 'neutral': 167}


Getting emb of batches:   0%|          | 0/3 [00:00<?, ?it/s]

Getting emb of batches: 100%|██████████| 3/3 [00:01<00:00,  2.32it/s]


In [55]:
selected_data[0]
len(selected_data)

501

# Tareget-unlabeled

In [56]:
import numpy as np

In [57]:
k=8
split='data/target-unlabeled'

In [58]:
def create_new_dataset(data_list, N = k):
    
    selected_samples = []

    if type(data_list[0]['label']) == list:
        labels = list(set([example['label'][0] for example in data_list]))

        print(f"Label Space: {labels}")
        count = {}
        for label in labels:
            count[label] = 0

        np.random.seed(0)
        chosen_indices = np.random.randint(low = 0, high = len(data_list), size = len(data_list))
    
        for chosen in chosen_indices:
            if (chosen not in selected_samples) and (count[data_list[chosen]['label'][0]] < (N/(len(count)))):
                selected_samples.append(chosen)
                count[data_list[chosen]['label'][0]] +=1

        selected_samples.sort()

        print(f"Count of selected labels: {count}")

        selected_data = list(np.array(data_list)[selected_samples])
        return selected_data, labels

    else:
        labels = list(set([example['label'] for example in data_list]))

        print(f"Label Space: {labels}")
        count = {}
        for label in labels:
            count[label] = 0

        np.random.seed(0)
        chosen_indices = np.random.randint(low = 0, high = len(data_list), size = len(data_list))

        for chosen in chosen_indices:

            if (chosen not in selected_samples) and (count[data_list[chosen]['label']] < (N/(len(count)))):
                selected_samples.append(chosen)
                count[data_list[chosen]['label']] +=1


        selected_samples.sort()
        print(f"Count of selected labels: {count}")
        selected_data = list(np.array(data_list)[selected_samples])
        return selected_data, labels

## Med-mcq

In [59]:
dataset_name = 'medmcqa'
dataset= load_dataset(dataset_name)['train']
test_id=[]
test_s=[]
test_l=[]

text2label={
    0:'A',
    1:'B',
    2:'C',
    3:'D'
}

for d in dataset:
    test_id.append(d['id'])
    test_l.append(text2label[d['cop']])

    q=d['question']+' \nA. '+d['opa']+' \nB. '+d['opb']+' \nC. '+d['opc']+' \nD. '+d['opd']

    test_s.append(q)

df={
    'id':test_id,
    'label':test_l,
    'sentence':test_s
}
test_df=pd.DataFrame(df)
test_s_df=test_df.to_dict('records')
print(len(test_s_df))

Using the latest cached version of the module from /home/eshaan/.cache/huggingface/modules/datasets_modules/datasets/medmcqa/f2fdfa9ccfbf9d148c0639e6afe3379f3c7e95c4d52d5e68ec1156e5004bd880 (last modified on Thu Sep 14 11:39:40 2023) since it couldn't be found locally at medmcqa., or remotely on the Hugging Face Hub.


182822


In [60]:
selected_data, labels = create_new_dataset(test_s_df)
prompt_temp_id='005'
create_embeddings(selected_data,prompter,prompt_temp_id,model)
data_to_save={
          'dataset_name': dataset_name,
          'prompt_temp_id':'005',
          'data': selected_data,
          'instruction': 'Given a multiple choice question containing four options "A.", "B.", "C." and "D." from a medical entrance exam. The question is related to a sub-field of medical science like Microbiology, Radiology, Ophthalmology, Surgery, Human anatomy, etc. Based on the question, the option and your knowledge of the medical field select the most appropriate answer from the provided choices "A.", "B.", "C." and "D.".',
          'labels': labels,
          'set':'target'
        }

with open(f'{split}/{dataset_name}_train.pkl', 'wb') as openfile:
        # Reading from json file
        pickle.dump(data_to_save, openfile)

Label Space: ['C', 'D', 'A', 'B']
Count of selected labels: {'C': 2, 'D': 2, 'A': 2, 'B': 2}


Getting emb of batches: 100%|██████████| 1/1 [00:00<00:00, 45.61it/s]


## sciq

In [61]:
dataset_name = 'sciq'
dataset= load_dataset(dataset_name)['train']
test_id=[]
test_s=[]
test_l=[]
id=1
import random
text2label={
    'A':'distractor3',
    'B':'distractor1',
    'C':'distractor2',
    'D':'correct_answer'
}

for d in dataset:

    k=['A','B','C','D']
    k_=['A','B','C','D']
    random.shuffle(k)
    q=d['question']   
    for i,l in zip(k,k_):
        op=text2label[i]
        if op=='correct_answer':
            test_l.append(l)

        q+=f'\n{l}. '+d[op]
                    



    test_id.append(id)
    id+=1
    test_s.append(q)

df={
    'id':test_id,
    'label':test_l,
    'sentence':test_s
}
test_df=pd.DataFrame(df)
test_s_df=test_df.to_dict('records')
print(len(test_s_df))

selected_data, labels = create_new_dataset(test_s_df)
prompt_temp_id='005'
create_embeddings(selected_data,prompter,prompt_temp_id,model)
data_to_save={
          'dataset_name': dataset_name,
          'prompt_temp_id':'005',
          'data': selected_data,
          'instruction': 'Given a question from a scientific exam about Physics, Chemistry, and Biology, among others. The question is in multiple choice format with four answer options "A.", "B.", "C." and "D.". Using your knowledge about the scientific fields answer the question and provide the label "A", "B", "C" and "D" as answer',
          'labels': labels,
          'set':'target'

}

with open(f'{split}/{dataset_name}_train.pkl', 'wb') as openfile:
        # Reading from json file
        pickle.dump(data_to_save, openfile)

Using the latest cached version of the module from /home/eshaan/.cache/huggingface/modules/datasets_modules/datasets/sciq/50e5c6e3795b55463819d399ec417bfd4c3c621105e00295ddb5f3633d708493 (last modified on Thu Sep 14 11:41:12 2023) since it couldn't be found locally at sciq., or remotely on the Hugging Face Hub.


11679
Label Space: ['C', 'D', 'A', 'B']
Count of selected labels: {'C': 2, 'D': 2, 'A': 2, 'B': 2}


Getting emb of batches: 100%|██████████| 1/1 [00:00<00:00, 49.17it/s]


## ARC-Challenge


In [62]:
dataset_name='ARC-Challenge'
dataset= load_dataset('ai2_arc',dataset_name)['train']
test_id=[]
test_s=[]
test_l=[]
id=1
import random

label2text={
    'A':'A',
    'B':'B',
    "C":"C",
    "D":"D",
    '2':'A',
    '1':'B',
    "3":"C",
    "4":"D",
}

for d in dataset:
    if len(d['choices']['text'])!=4:
        continue
    test_id.append(d['id'])
    id+=1
    test_s.append(d['question']+'\nA. '+d['choices']['text'][0]+'\nB. '+d['choices']['text'][1]+'\nC. '+d['choices']['text'][2]+'\nD. '+d['choices']['text'][3])
    test_l.append(label2text[d['answerKey']])

df={
    'id':test_id,
    'label':test_l,
    'sentence':test_s
}
test_df=pd.DataFrame(df)
test_s_df=test_df.to_dict('records')
print(len(test_s_df))
selected_data, labels = create_new_dataset(test_s_df)
prompt_temp_id='005'
create_embeddings(test_s_df,prompter,prompt_temp_id,model)
data_to_save={
        'dataset_name':dataset_name,
        'prompt_temp_id':'005',
        'data':selected_data,
        'instruction':'Given a question answering task from the 3rd to 9th-grade science exam. The question contains four options "A.", "B.", "C." and "D." Select the most appropriate choice that answers the question',
        'labels': labels,
        'set':'target'
        }

with open(f'{split}/{dataset_name}_train.pkl', 'wb') as openfile:
        # Reading from json file
        pickle.dump(data_to_save, openfile)


Using the latest cached version of the module from /home/eshaan/.cache/huggingface/modules/datasets_modules/datasets/ai2_arc/1569c2591ea2683779581d9fb467203d9aa95543bb9b75dcfde5da92529fd7f6 (last modified on Fri Aug 11 17:17:10 2023) since it couldn't be found locally at ai2_arc., or remotely on the Hugging Face Hub.


1117
Label Space: ['C', 'D', 'A', 'B']
Count of selected labels: {'C': 2, 'D': 2, 'A': 2, 'B': 2}


Getting emb of batches: 100%|██████████| 2/2 [00:00<00:00,  2.48it/s]


{'id': 'Mercury_7216878',
 'label': 'C',
 'sentence': 'Some health care professionals recommend that children use insect repellent before going outside. Using insect repellent is a good way to keep the insects from\nA. reproducing.\nB. laying eggs.\nC. spreading disease.\nD. dying.',
 'emb': tensor([-1.2950e-03,  4.3211e-02,  1.1702e-02,  3.7854e-02,  8.7574e-02,
          1.0261e-01,  1.4885e-01, -1.2478e-02, -1.3461e-02,  4.7728e-02,
          1.3910e-01,  3.8054e-02, -5.0591e-02,  8.3565e-02,  1.7855e-02,
          1.2662e-01,  1.3615e-01,  2.2474e-02, -3.0715e-02, -9.3841e-02,
         -2.4924e-02,  3.3524e-02,  4.3234e-02, -2.8324e-02, -4.4351e-02,
          1.0436e-02, -3.8043e-02, -1.4960e-02,  4.3807e-02, -3.7751e-02,
          2.7026e-02, -1.8748e-03, -9.1333e-02, -5.5277e-02, -6.3995e-02,
          5.3490e-02,  2.3262e-03, -2.8691e-02, -4.1016e-02,  3.7887e-02,
         -6.9167e-03,  5.6006e-02, -1.4653e-01, -3.0376e-02, -7.6202e-02,
         -3.8039e-02,  6.8713e-02,  5.9085

## Social_i_qa


In [63]:
dataset_name = 'social_i_qa'
dataset= load_dataset(dataset_name)['train']
test_id=[]
test_s=[]
test_l=[]
test_c=[]
id=1

text2label={
    '1':'A',
    '2':'B',
    '3':'C'
}
label2text={
    1:'answerA',
    2:'answerB',
    3:'answerC'
}


for d in dataset:
          test_id.append(id)
          id+=1

          test_l.append(text2label[d['label']])
          test_c.append(d['context'])

          q=d['question']+' \nA. '+d['answerA']+' \nB. '+d['answerB']+' \nC. '+d['answerC']
          test_s.append(q)

df={
    'id':test_id,
    'label':test_l,
    'sentence':test_s,
    'context':test_c
}
test_df=pd.DataFrame(df)
test_s_df=test_df.to_dict('records')
print(len(test_s_df))
selected_data, labels = create_new_dataset(test_s_df)

prompt_temp_id='003'
create_embeddings(test_s_df,prompter,prompt_temp_id,model)
data_to_save={
          'dataset_name':dataset_name,
          'prompt_temp_id':'003',
          'data': selected_data,
          'instruction': 'Given an action as the context and a related question, you are to answer the question based on the context using your social intelligence. The question is of multiple choice form with three options "A", "B" and "C". Select the most appropriate answer from the provided choices "A", "B" and "C".',
          'labels': labels,
          'set':'target'
          
          }

with open(f'{split}/{dataset_name}_train.pkl', 'wb') as openfile:
        # Reading from json file
        pickle.dump(data_to_save, openfile)

Using the latest cached version of the module from /home/eshaan/.cache/huggingface/modules/datasets_modules/datasets/social_i_qa/674d85e42ac7430d3dcd4de7007feaffcb1527c535121e09bab2803fbcc925f8 (last modified on Thu Sep 14 11:41:32 2023) since it couldn't be found locally at social_i_qa., or remotely on the Hugging Face Hub.


33410
Label Space: ['C', 'A', 'B']
Count of selected labels: {'C': 3, 'A': 3, 'B': 3}


Getting emb of batches: 100%|██████████| 33/33 [00:19<00:00,  1.71it/s]


{'id': 2733,
 'label': 'A',
 'sentence': "What does Robin need to do before this? \nA. make sure Riley doesn't fall off the ladder \nB. be prepared for them to fall \nC. safe",
 'context': "Robin supported Riley's weight to help hold them steady on the ladder.",
 'emb': tensor([-6.4449e-02,  1.7841e-02,  3.6968e-02,  1.9502e-02,  1.4299e-02,
          5.4664e-02,  3.8244e-02,  1.1291e-01, -7.5707e-02,  4.5942e-02,
          3.1504e-02, -3.5769e-02, -2.0830e-02,  2.1833e-02, -4.6969e-02,
          7.0748e-02, -1.7893e-02,  5.3615e-02, -2.9572e-02,  2.7360e-02,
         -2.9490e-02, -1.0689e-01,  3.9402e-02,  9.0853e-02, -5.0609e-02,
         -1.5155e-02, -7.8933e-02, -1.3846e-02,  3.3759e-02, -1.3335e-02,
          7.6841e-03, -4.0268e-02,  1.8667e-02,  2.1241e-02, -4.5002e-02,
          4.7769e-02,  4.2563e-02,  1.8124e-02, -3.0843e-02, -8.2606e-03,
          7.7016e-03, -1.8081e-02, -4.8170e-02,  2.8050e-03,  2.6291e-02,
          5.8090e-02, -2.9885e-02,  5.0821e-02,  2.7274e-02,  2.

## Financial Pharasebank


In [64]:
dataset_name='financial_phrasebank'
dataset= load_dataset(dataset_name,'sentences_allagree')['train']
dataset=dataset.train_test_split(test_size=0.2, stratify_by_column="label")
dataset=dataset['test']
test_id=[]
test_s=[]
test_l=[]
id=1

text2label={
    1:'neutral',
    2:'positive',
    0:'negative'
}

for d in dataset:
          test_id.append(id)
          id+=1
          test_s.append(d['sentence'])
          test_l.append(text2label[d['label']])

df={
    'id':test_id,
    'label':test_l,
    'sentence':test_s
}
test_df=pd.DataFrame(df)
test_s_df=test_df.to_dict('records')
print(len(test_s_df))
selected_data, labels = create_new_dataset(test_s_df)

prompt_temp_id='004'
create_embeddings(test_s_df,prompter,prompt_temp_id,model)
data_to_save={
        'dataset_name': dataset_name,
        'prompt_temp_id':'004',
        'data': selected_data,
        'instruction': 'Given a sentence mined from a financial news article, you are to determine the sentiment polarity of the sentence. The task deals with financial sentiment analysis. Based on the sentiment conveyed by the sentence, label the sentence as "negative", "positive" or "neutral"',
        'labels': labels,
        'set':'target'

}

with open(f'{split}/{dataset_name}_train.pkl', 'wb') as openfile:
        # Reading from json file
        pickle.dump(data_to_save, openfile)

Using the latest cached version of the module from /home/eshaan/.cache/huggingface/modules/datasets_modules/datasets/financial_phrasebank/550bde12e6c30e2674da973a55f57edde5181d53f5a5a34c1531c53f93b7e141 (last modified on Fri Aug 11 17:37:50 2023) since it couldn't be found locally at financial_phrasebank., or remotely on the Hugging Face Hub.


2264
Label Space: ['negative', 'positive', 'neutral']
Count of selected labels: {'negative': 3, 'positive': 3, 'neutral': 3}


Getting emb of batches: 100%|██████████| 3/3 [00:01<00:00,  2.31it/s]


9